# HW

Get statistic from MedHistory

## Read Data

In [133]:
import re
import os
import random
import math
import json
import time
import pandas as pd
import numpy as np
import copy as copy
import matplotlib.pyplot as plt
import translators as ts
import translators.server as tss
from translate import Translator
from pandas import Timestamp
from datetime import datetime
from os import getcwd
from os.path import join
%matplotlib inline


In [134]:
data_path_1 = os.path.join(
    os.getcwd().rstrip('src'), 'data', 'TaiwanDeathDetail2021_ts.csv')
data_path_2 = os.path.join(
    os.getcwd().rstrip('src'), 'data', 'TaiwanDeathDetail20220611_ts.csv')
data1 = pd.read_csv(data_path_1, encoding='utf-8')
data2 = pd.read_csv(data_path_2, encoding='utf-8')
data1.head()

,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022/12/19,4397,M,60.5,有,N,2022/05/22,發燒,2022/05/22,2022/05/23,2022/05/24,2022/12/01
1,2022/12/15,10511,M,50.5,有,Y,2022/06/03,呼吸喘,2022/06/03,2022/06/04,2022/06/05,2022/11/19
2,2022/11/09,8724,F,90.5,有,Y,2022/05/24,發燒、喉嚨痛,2022/05/31,2022/05/30,2022/06/01,2022/10/11
3,2022/10/25,12235,M,80.5,有,NaN,2022/06/08,食慾不振、昏睡,2022/06/08,2022/05/31,2022/06/10,2022/10/22
4,2022/10/09,8748,F,70.5,有,NaN,NaN,--,2022/05/30,2022/06/24,2022/06/01,2022/10/04


In [135]:
data2.head()

,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,ConfirmDate,DeceaseDate
0,2022/06/11,NaN,M,90.5,慢性肺病,3.0,2022/05/18,發燒,NaN,NaN,2022/05/18,2022/06/06
1,2022/06/11,NaN,F,90.5,高血壓等,0.0,2022/06/01,發燒、咳嗽、呼吸喘,NaN,NaN,2022/06/01,2022/06/07
2,2022/06/11,NaN,F,90.5,心血管疾病,0.0,2022/05/30,咳嗽、呼吸困難、顫抖,NaN,NaN,2022/05/30,2022/06/06
3,2022/06/11,NaN,F,90.5,心血管疾病等,1.0,NaN,調查中,NaN,NaN,2022/06/06,2022/06/07
4,2022/06/11,NaN,F,90.5,神經系統疾病等,3.0,2022/05/20,呼吸困難、血氧下降、胸部X光片肺部 雙側浸潤,NaN,NaN,2022/05/22,2022/05/21


## Convert Column 'Symptoms'

In [136]:
def GetFeatures(PdSeries):
    # PdSeries: pandas series, not list
    # Use '@' as separator for intermediate results
    # remove white spaces, replace ',', ';', ... with feature separator '、'
    FeatureNames = []
    FeatureBeforeSplit = []
    string = copy.copy(PdSeries)
    string = string.fillna('無症狀')
    
    for i in string:
        i = i.replace(' ', '')
        i = i.replace('\n', '')
        i = i.replace(', ', '、')
        i = i.replace('; ', '、')
        i = i.replace('；', '、')
        i = i.replace('，', '、')
        i = i.replace(',', '、')
        i = i.replace('及', '、')
        i = i.replace('和', '、')
        i = re.sub(r'[0-9/]', '', i)
        i = i.strip('等')
        i = i.strip('--')

        FeatureNames = FeatureNames + i.split('、')
        FeatureBeforeSplit = FeatureBeforeSplit + [i]

    FeatureNames = set(FeatureNames)
    FeatureBeforeSplit = set(FeatureBeforeSplit)
    FeatureNames.discard('')
    FeatureBeforeSplit.discard('')
    
    return FeatureNames, FeatureBeforeSplit


In [137]:
def GetFeatureOccurrence(DFin, ConsolidatedFeatures, DictSymptom):
    DF = copy.copy(DFin)
    for i in ConsolidatedFeatures:
        DF[i] = 0 # create a new column for each ConsolidatedFeature
    for i in range(DF.shape[0]):
        Features, FeaturesBeforeSplit = GetFeatures(data1['Symptoms'].iloc[i:i+1])
        for j in Features:
            DF.at[i, DictSymptom[j]] = 1

    return DF

### Dataset 1

#### Get Unique Symptoms

In [138]:
SympDictOrigin_1 = set(data1['Symptoms'])
SympName_1, FeatureBeforeSplit_1 = GetFeatures(data1['Symptoms'])
SympName_1

{'上呼吸道症狀',
 '不順',
 '乾咳',
 '低燒',
 '低血壓',
 '低血氧',
 '倦怠',
 '入境後出現發燒',
 '全身不舒服',
 '全身倦怠',
 '全身無力',
 '全身痠痛',
 '全身痠軟',
 '全身虛弱',
 '全身虛弱無力',
 '出現發燒',
 '口乾舌燥',
 '吐血',
 '味覺喪失',
 '味覺異常',
 '呼吸不順',
 '呼吸喘',
 '呼吸困難',
 '呼吸急促',
 '呼吸短促',
 '呼吸窘迫',
 '呼吸道症狀',
 '咳嗽',
 '咳嗽不適',
 '咳嗽有痰',
 '咳嗽症狀',
 '喉嚨不適',
 '喉嚨刺激感',
 '喉嚨痛',
 '喉嚨癢',
 '喉痛',
 '喘',
 '喘鳴',
 '嗅味覺喪失',
 '嗅味覺異常',
 '嗅覺異常',
 '嗜睡',
 '嗽',
 '嘔吐',
 '嘔吐感',
 '噁心',
 '噁心感',
 '四肢無力',
 '因其他原因就醫',
 '困難',
 '寒顫',
 '左側肢體無力',
 '心悸',
 '心情不佳無食慾',
 '心臟不舒服',
 '意識不清',
 '意識改變',
 '感冒症狀',
 '手抖',
 '抽蓄',
 '採撿時無症狀',
 '採檢時無症狀',
 '採檢時無相關症狀',
 '採檢時無相關症狀呼吸急促',
 '昏睡',
 '有痰',
 '氣喘',
 '沒有食慾',
 '沒精神',
 '活動差',
 '流鼻水',
 '流鼻涕',
 '無症狀',
 '無症狀（接觸者採檢陽轉）',
 '猝死',
 '畏冷',
 '畏寒',
 '疲倦',
 '疲憊',
 '痰有血絲',
 '發冷',
 '發燒',
 '眼球上吊',
 '篩檢時無症狀',
 '精神倦怠',
 '肌肉痠痛',
 '肌肉酸痛',
 '肢體變黑',
 '肩膀痠痛',
 '肺炎',
 '胸悶',
 '胸痛',
 '胸部不適感',
 '腸胃不舒服',
 '腸胃道不適',
 '腹瀉',
 '腹痛',
 '腹部不適',
 '自覺體溫偏高',
 '虛弱',
 '虛弱無力',
 '血氧降低',
 '調查中',
 '走路無力',
 '身體不適',
 '身體無力',
 '身體痠痛',
 '輕微咳嗽',
 '關節痛',
 '關節肌肉痛',
 '隔離期間呼吸喘',
 '頭昏',
 '頭暈',
 '頭暈目眩',
 '頭痛',

In [139]:
FeatureBeforeSplit_1


{'上呼吸道症狀、發燒',
 '乾咳、發燒、呼吸急促',
 '低血氧',
 '入境後出現發燒、全身倦怠',
 '全身不舒服、呼吸喘、咳嗽、精神倦怠',
 '全身倦怠',
 '全身倦怠、咳嗽、食慾不振',
 '全身倦怠、發燒、乾咳',
 '全身倦怠、發燒、咳嗽',
 '全身無力',
 '全身無力、咳嗽',
 '全身無力、咳嗽有痰、痰有血絲、呼吸喘',
 '全身無力、畏寒',
 '全身痠痛、呼吸喘',
 '全身痠痛、呼吸困難、發燒、咳嗽有痰',
 '全身痠痛、咳嗽',
 '全身虛弱、咳嗽、呼吸喘',
 '口乾舌燥、疲倦',
 '味覺異常、呼吸急促、食慾不佳',
 '味覺異常、咳嗽、發燒、呼吸喘',
 '呼吸喘',
 '呼吸喘、不順',
 '呼吸喘、全身無力',
 '呼吸喘、全身虛弱',
 '呼吸喘、呼吸困難',
 '呼吸喘、咳嗽',
 '呼吸喘、咳嗽、發燒',
 '呼吸喘、寒顫',
 '呼吸喘、有痰',
 '呼吸喘、疲倦、身體無力',
 '呼吸喘、發燒',
 '呼吸喘、肢體變黑',
 '呼吸喘、腹瀉',
 '呼吸喘、食慾不振',
 '呼吸喘、食慾不振、全身倦怠',
 '呼吸困難',
 '呼吸困難、全身倦怠',
 '呼吸困難、呼吸喘',
 '呼吸困難、呼吸喘、腹瀉',
 '呼吸困難、呼吸短促',
 '呼吸困難、咳嗽',
 '呼吸困難、咳嗽、喉嚨痛、發燒、腹瀉、嘔吐、頭痛',
 '呼吸困難、咳嗽、發燒',
 '呼吸困難、咳嗽、食慾不振',
 '呼吸困難、咳嗽有痰',
 '呼吸困難、喘',
 '呼吸困難、意識不清',
 '呼吸困難、發燒',
 '呼吸困難、發燒、咳嗽、全身倦怠',
 '呼吸困難、發燒、嘔吐、噁心',
 '呼吸困難、發燒、腹瀉',
 '呼吸困難、發燒、關節痛',
 '呼吸困難、肺炎',
 '呼吸急促',
 '呼吸急促、低燒',
 '呼吸急促、呼吸喘',
 '呼吸急促、肌肉痠痛',
 '呼吸急促、肺炎',
 '呼吸短促',
 '呼吸短促、呼吸喘',
 '呼吸短促、呼吸喘、食慾不振、發燒',
 '呼吸短促、呼吸困難、發燒、咳嗽',
 '呼吸短促、咳嗽',
 '呼吸窘迫',
 '呼吸道症狀',
 '咳嗽',
 '咳嗽、低血壓',
 '咳嗽、全身倦怠、沒有食慾',
 '咳嗽、全身倦怠、發燒',
 '咳嗽、全身無力',
 '咳嗽、全身無力、腹瀉',
 '咳嗽、呼吸喘',


#### Consolidate Symptoms

In [140]:
SympNameDict_1 = {
    '上呼吸道症狀': '上呼吸道症狀',
    '不順': '未知',
    '乾咳': '咳嗽',
    '低燒': '發燒',
    '低血壓': '低血壓',
    '低血氧': '低血氧',
    '倦怠': '倦怠虛弱',
    '入境後出現發燒': '發燒',
    '全身不舒服': '倦怠虛弱',
    '全身倦怠': '倦怠虛弱',
    '全身無力': '倦怠虛弱',
    '全身痠痛': '關節肌肉痛',
    '全身痠軟': '關節肌肉痛',
    '全身虛弱': '倦怠虛弱',
    '全身虛弱無力': '倦怠虛弱',
    '出現發燒': '發燒',
    '口乾舌燥': '口乾舌燥',
    '吐血': '吐血',
    '味覺喪失': '嗅味覺異常',
    '味覺異常': '嗅味覺異常',
    '呼吸不順': '呼吸困難',
    '呼吸喘': '呼吸困難',
    '呼吸困難': '呼吸困難',
    '呼吸急促': '呼吸困難',
    '呼吸短促': '呼吸困難',
    '呼吸窘迫': '呼吸困難',
    '呼吸道症狀': '呼吸困難',
    '咳嗽': '咳嗽',
    '咳嗽不適': '咳嗽',
    '咳嗽有痰': '咳嗽',
    '咳嗽症狀': '咳嗽',
    '喉嚨不適': '咳嗽',
    '喉嚨刺激感': '咳嗽',
    '喉嚨痛': '咳嗽',
    '喉嚨癢': '咳嗽',
    '喉痛': '咳嗽',
    '喘': '呼吸困難',
    '喘鳴': '呼吸困難',
    '嗅味覺喪失': '嗅味覺異常',
    '嗅味覺異常': '嗅味覺異常',
    '嗅覺異常': '嗅味覺異常',
    '嗜睡': '倦怠虛弱',
    '嗽': '咳嗽',
    '嘔吐': '噁心嘔吐',
    '嘔吐感': '噁心嘔吐',
    '噁心': '噁心嘔吐',
    '噁心感': '噁心嘔吐',
    '四肢無力': '倦怠虛弱',
    '因其他原因就醫': '未知',
    '困難': '未知',
    '寒顫': '畏寒',
    '左側肢體無力': '倦怠虛弱',
    '心悸': '心臟不適',
    '心情不佳無食慾': '食慾差',
    '心臟不舒服': '心臟不適',
    '意識不清': '意識不清',
    '意識改變': '意識不清',
    '感冒症狀': '感冒症狀',
    '手抖': '手抖',
    '抽蓄': '抽蓄',
    '採撿時無症狀': '無症狀',
    '採檢時無症狀': '無症狀',
    '採檢時無相關症狀': '無症狀',
    '採檢時無相關症狀呼吸急促': '呼吸困難',
    '昏睡': '倦怠虛弱',
    '有痰': '有痰',
    '氣喘': '呼吸困難',
    '沒有食慾': '食慾差',
    '沒精神': '倦怠虛弱',
    '活動差': '倦怠虛弱',
    '流鼻水': '流鼻水鼻涕鼻塞',
    '流鼻涕': '流鼻水鼻涕鼻塞',
    '無症狀': '無症狀',
    '無症狀（接觸者採檢陽轉）': '無症狀',
    '猝死': '猝死',
    '畏冷': '畏寒',
    '畏寒': '畏寒',
    '疲倦': '倦怠虛弱',
    '疲憊': '倦怠虛弱',
    '痰有血絲': '有痰',
    '發冷': '畏寒',
    '發燒': '發燒',
    '眼球上吊': '眼球上吊',
    '篩檢時無症狀': '無症狀',
    '精神倦怠': '倦怠虛弱',
    '肌肉痠痛': '關節肌肉痛',
    '肌肉酸痛': '關節肌肉痛',
    '肢體變黑': '肢體變黑',
    '肩膀痠痛': '關節肌肉痛',
    '肺炎': '肺炎',
    '胸悶': '胸悶',
    '胸痛': '胸悶',
    '胸部不適感': '胸悶',
    '腸胃不舒服': '腸胃不適',
    '腸胃道不適': '腸胃不適',
    '腹瀉': '腸胃不適',
    '腹痛': '腸胃不適',
    '腹部不適': '腸胃不適',
    '自覺體溫偏高': '發燒',
    '虛弱': '倦怠虛弱',
    '虛弱無力': '倦怠虛弱',
    '血氧降低': '低血氧',
    '調查中': '未知',
    '走路無力': '倦怠虛弱',
    '身體不適': '倦怠虛弱',
    '身體無力': '倦怠虛弱',
    '身體痠痛': '關節肌肉痛',
    '輕微咳嗽': '咳嗽',
    '關節痛': '關節肌肉痛',
    '關節肌肉痛': '關節肌肉痛',
    '隔離期間呼吸喘': '呼吸困難',
    '頭昏': '頭暈',
    '頭暈': '頭暈',
    '頭暈目眩': '頭暈',
    '頭痛': '頭暈',
    '食慾下降': '食慾差',
    '食慾不佳': '食慾差',
    '食慾不振': '食慾差',
    '食慾差': '食慾差',
    '體力不支': '倦怠虛弱',
    '體力差': '倦怠虛弱',
    '體力變差': '倦怠虛弱',
    '鼻塞': '流鼻水鼻涕鼻塞',
    }


In [141]:
len(set(SympNameDict_1.values()))


30

In [142]:
ConsolidatedSymp_1 = set(SympNameDict_1.values())
ConsolidatedSymp_1 - SympName_1

{'倦怠虛弱', '噁心嘔吐', '心臟不適', '未知', '流鼻水鼻涕鼻塞', '腸胃不適'}

In [143]:
len(SympName_1 - ConsolidatedSymp_1)

99

In [144]:
SympNameDict_1.items()

dict_items([('上呼吸道症狀', '上呼吸道症狀'), ('不順', '未知'), ('乾咳', '咳嗽'), ('低燒', '發燒'), ('低血壓', '低血壓'), ('低血氧', '低血氧'), ('倦怠', '倦怠虛弱'), ('入境後出現發燒', '發燒'), ('全身不舒服', '倦怠虛弱'), ('全身倦怠', '倦怠虛弱'), ('全身無力', '倦怠虛弱'), ('全身痠痛', '關節肌肉痛'), ('全身痠軟', '關節肌肉痛'), ('全身虛弱', '倦怠虛弱'), ('全身虛弱無力', '倦怠虛弱'), ('出現發燒', '發燒'), ('口乾舌燥', '口乾舌燥'), ('吐血', '吐血'), ('味覺喪失', '嗅味覺異常'), ('味覺異常', '嗅味覺異常'), ('呼吸不順', '呼吸困難'), ('呼吸喘', '呼吸困難'), ('呼吸困難', '呼吸困難'), ('呼吸急促', '呼吸困難'), ('呼吸短促', '呼吸困難'), ('呼吸窘迫', '呼吸困難'), ('呼吸道症狀', '呼吸困難'), ('咳嗽', '咳嗽'), ('咳嗽不適', '咳嗽'), ('咳嗽有痰', '咳嗽'), ('咳嗽症狀', '咳嗽'), ('喉嚨不適', '咳嗽'), ('喉嚨刺激感', '咳嗽'), ('喉嚨痛', '咳嗽'), ('喉嚨癢', '咳嗽'), ('喉痛', '咳嗽'), ('喘', '呼吸困難'), ('喘鳴', '呼吸困難'), ('嗅味覺喪失', '嗅味覺異常'), ('嗅味覺異常', '嗅味覺異常'), ('嗅覺異常', '嗅味覺異常'), ('嗜睡', '倦怠虛弱'), ('嗽', '咳嗽'), ('嘔吐', '噁心嘔吐'), ('嘔吐感', '噁心嘔吐'), ('噁心', '噁心嘔吐'), ('噁心感', '噁心嘔吐'), ('四肢無力', '倦怠虛弱'), ('因其他原因就醫', '未知'), ('困難', '未知'), ('寒顫', '畏寒'), ('左側肢體無力', '倦怠虛弱'), ('心悸', '心臟不適'), ('心情不佳無食慾', '食慾差'), ('心臟不舒服', '心臟不適'), ('意識不清', '意識不清'), ('意識改變', '意識不清'), ('感冒症狀', '感冒症狀

In [145]:
data1 = GetFeatureOccurrence(data1, ConsolidatedSymp_1, SympNameDict_1)
data1.head()

,Date,CaseID,Gender,Age,MedHistory,Vac,SickDate,Symptoms,SpecimenDate,QuarantineDate,...,發燒,口乾舌燥,嗅味覺異常,畏寒,流鼻水鼻涕鼻塞,感冒症狀,吐血,腸胃不適,頭暈,有痰
0,2022/12/19,4397,M,60.5,有,N,2022/05/22,發燒,2022/05/22,2022/05/23,...,1,0,0,0,0,0,0,0,0,0
1,2022/12/15,10511,M,50.5,有,Y,2022/06/03,呼吸喘,2022/06/03,2022/06/04,...,0,0,0,0,0,0,0,0,0,0
2,2022/11/09,8724,F,90.5,有,Y,2022/05/24,發燒、喉嚨痛,2022/05/31,2022/05/30,...,1,0,0,0,0,0,0,0,0,0
3,2022/10/25,12235,M,80.5,有,NaN,2022/06/08,食慾不振、昏睡,2022/06/08,2022/05/31,...,0,0,0,0,0,0,0,0,0,0
4,2022/10/09,8748,F,70.5,有,NaN,NaN,--,2022/05/30,2022/06/24,...,0,0,0,0,0,0,0,0,0,0


In [146]:
data1SympOccur = data1[ConsolidatedSymp_1].sum().reset_index()
data1SympOccur.columns = ['Symptoms', 'Occurrence']
data1SympOccur.sort_values(by='Occurrence', ascending=False, inplace=True)
data1SympOccur.reset_index(inplace=True, drop=True)
data1SympOccur.head()


C:\Users\dachu\AppData\Local\Temp\ipykernel_19972\2921420921.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  data1SympOccur = data1[ConsolidatedSymp_1].sum().reset_index()


,Symptoms,Occurrence
0,發燒,409
1,咳嗽,352
2,呼吸困難,291
3,倦怠虛弱,99
4,無症狀,92


#### Export Converted Dataset

In [147]:
data_path_1 = os.path.join(
    os.getcwd().rstrip('src'), 'data', 'TaiwanDeathDetail2021_ts1.csv')
data1.to_csv(data_path_1, encoding='utf-8')


### Dataset 2

#### Get unique Symptoms

In [148]:
SympDictOrigin_2 = set(data2['Symptoms'])
SympName_2, FeatureBeforeSplit_2 = GetFeatures(data2['Symptoms'])
SympName_2

{'CXR有肋膜積水',
 'CXR顯示兩',
 'CXR顯示兩側輕微肺浸潤',
 'CXR顯示肺炎',
 'SOB',
 'SpO',
 'X光片顯示肺炎',
 '上呼吸道感染症狀',
 '上呼吸道症狀',
 '上腸胃道出血',
 '上腹部痛',
 '下肢水腫',
 '下肢無力',
 '不自主運動',
 '不適',
 '乾咳',
 '休克',
 '低燒',
 '低白蛋白血症',
 '低血壓',
 '低血氧',
 '低血氧(%)',
 '低血氧(SpO%)',
 '住院期間採檢確診',
 '便秘',
 '個案路倒送醫',
 '倦怠',
 '倦怠無力',
 '偶咳',
 '僵硬抖動',
 '全身不適',
 '全身倦怠',
 '全身冒冷汗',
 '全身水腫',
 '全身無力',
 '全身虛弱',
 '全身虛弱無力',
 '冒冷汗',
 '到院前OHCA',
 '到院前停止呼吸心跳',
 '到院前死亡',
 '到院前無呼吸心跳',
 '到院前無生命徵象',
 '反應性下降',
 '反應遲鈍',
 '口水多',
 '叫不醒',
 '右上腹痛',
 '右下腹痛',
 '吃不下',
 '吐',
 '吞嚥',
 '呼吸不順',
 '呼吸喘',
 '呼吸喘(於居住機構死亡後採檢確診)',
 '呼吸喘.血氧低',
 '呼吸喘促',
 '呼吸喘咳嗽',
 '呼吸喘快',
 '呼吸喘急',
 '呼吸喘痰多',
 '呼吸困難',
 '呼吸困難(SpO%)',
 '呼吸困難且低血氧',
 '呼吸困難日',
 '呼吸急促',
 '呼吸急促且有痰',
 '呼吸淺快',
 '呼吸淺快入急診時使用OMASKLMIN',
 '呼吸短促',
 '呼吸窘迫',
 '呼吸胸悶',
 '呼吸衰竭',
 '呼吸費力',
 '呼吸音異常',
 '咳嗽',
 '咳嗽(到院前無生命徵象)',
 '咳嗽(於家中昏迷送醫',
 '咳嗽有痰',
 '咳嗽有血絲',
 '咳嗽痰多',
 '咳痰',
 '咳痰不順',
 '咳血',
 '咳血(到院前無生命徵象)',
 '喉嚨不適',
 '喉嚨乾',
 '喉嚨卡卡',
 '喉嚨有痰',
 '喉嚨痛',
 '喉嚨痛\xa0全身倦怠',
 '喉嚨癢',
 '喘',
 '喘不過氣',
 '喟嘆式呼吸',
 '喪失意識',
 '嗜睡',
 '嘔吐',
 '嘔吐不適',
 '嘔吐且意識改變

#### Consolidate Symptoms